# The size and shape of a CSV file

<div class="alert alert-info"><img src="../images/hhicon.png" width="50px" style="vertical-align: bottom; margin-right: 10px;">Remember to run each cell using <b>Shift+Enter</b>!</div>

## Opening a CSV file

In [ ]:
import pandas as pd
import altair as alt

Using Pandas we can open a CSV file from a url – no need to download it first. To begin with, we open the CSV file and give it the name `df` (for 'DataFrame').

In this case, we're opening the [Brisbane Hospital registers of deaths 1899 to 1913](https://data.qld.gov.au/dataset/34ff0384-e09a-4ba0-8ea4-f54534e1aace) from the Queensland State Archives.

In [ ]:
df = pd.read_csv('https://www.data.qld.gov.au/dataset/34ff0384-e09a-4ba0-8ea4-f54534e1aace/resource/265df65a-372e-4f90-a6b8-5a47c30fc740/download/brisbane-hospital-deaths-1899-1913.csv')

From now on we can just use `df` when we want to refer to the dataset. 

Pandas dataframes have lots of useful built-in tricks for exploring and analysing the dataset. We can use `.head()` to show us the first few rows.

In [ ]:
df.head()

<div class="alert alert-info"><img src="../images/hhicon.png" width="50px" style="vertical-align: bottom; margin-right: 10px;">Try putting a number in the brackets after <code>.head</code>. What happens? (Remember to hit <b>Shift+Enter</b> to run the cell once you've changed it!)</div>

## How big is the dataset?

We can use `.shape` to give us the number of rows and columns in the dataset.

In [ ]:
df.shape

So this dataset has 4,866 rows.

## Get the values of a single column

To get the values of a single column (what Pandas calls a 'series'), just supply the column name in square brackets.

In [ ]:
df['Last name']

Note that for ease of display, Pandas shows you just the first and last values.

## Get the values of a single row

Rows in a dataframe are indexed – they're the numbers you can see in the left hand column. You can access any row by supplying it's index number to `.iloc`.

In [ ]:
df.iloc[4]

<div class="alert alert-info"><img src="../images/hhicon.png" width="50px" style="vertical-align: bottom; margin-right: 10px;">Try changing the number in the square brackets! What happens?</div>

## Get all the rows that match a particular value

We can also supply conditions to `.iloc` to filter rows by a particular column and value. So to find all the rows where the `Last name` was 'ABRAHAM'...

In [ ]:
df.loc[df['Last name'] == 'ABRAHAM']

<div class="alert alert-info"><img src="../images/hhicon.png" width="50px" style="vertical-align: bottom; margin-right: 10px;">Try changing the value of the name – what about 'SMITH'? Remember to keep the quotes around the name.</div>

## Find (or exclude) missing values

To find all rows where `Age` is not available, we can use `.iloc` with `.isnull`.

In [ ]:
df.loc[df['Age'].isnull()]

So there are 334 rows that don't have a value for `Age`. Note that Pandas represents a missing value as `NaN`.

If we want to get all the available values for `Age` (without the `NaN`s), we can use `.dropna`.

In [ ]:
df['Age'].dropna()

## Find the youngest and oldest ages

Using `.min()` and `.max()` we can find the smallest and largest values in a field.

In [ ]:
df['Age'].min()

In [ ]:
df['Age'].max()

To find the index of the row with the largest value we can use `.idxmax()`. Then we can give that value to `.iloc` to get the details of that row. So who was the 96-year-old patient?

In [ ]:
df.loc[df['Age'].idxmax()]

We can even find out the mean age at death – we just use `.mean()`.

In [ ]:
df['Age'].mean()

<div class="alert alert-info"><img src="../images/hhicon.png" width="50px" style="vertical-align: bottom; margin-right: 10px;">A challenge! Can you use what you've learnt on this page to find all records where the age at death was recorded as 0? The answer is hidden below – click on the dots to reveal it.</div>

In [ ]:
df.loc[df['Age'] == 0]

Finally, let's chart the distribution of ages.

In [ ]:
alt.Chart(df.loc[df['Age'].notnull()]).mark_bar().encode(
    alt.X("Age:Q", bin=True),
    y='count()',
    tooltip=[alt.Tooltip('Age', bin=True), 'count()']
)

## What format are the dates in?

The 'Date of death column' looks pretty consistent, with a standard ISO YYYY-MM-DD format.

In [ ]:
df['Date of death']

If it is as consistent as it seems, we should be able to convert these date strings in Python Datetime objects. Once again, Pandas makes this easy with the `.to_datetime` function. We'll set `errors` to 'coerce'. That means that if any dates can't be converted, they'll be given a null value.

In [ ]:
df['date'] = pd.to_datetime(df['Date of death'], errors='coerce', utc=True)

Let's have a look at the results.

In [ ]:
df['date']

Looks like the strings were successfully converted to dates, but were there any problems. Let's check the null values.

In [ ]:
df.loc[df['date'].isnull()]

So two rows had bad dates – one was missing, and one was in a non-standard format. Let's create a chart to view the results. First we'll reorganise the data a bit to get the total number of deaths per year.

In [ ]:
deaths_by_year = df['date'].dt.year.value_counts().to_frame().reset_index()
deaths_by_year.columns = ['year', 'deaths']
deaths_by_year['year'] = deaths_by_year['year'].astype(int)
deaths_by_year.head()

Then we'll build the chart.

In [ ]:
alt.Chart(deaths_by_year).mark_bar().encode(
    x='year:N',
    y='deaths:Q',
    tooltip=['year:N', 'deaths']
).properties(width=600)

## What's next?

Using this notebook as a template, try exploring the size and shape of a different CSV file. There's plenty to choose from in this [GLAM data list](https://glam-workbench.github.io/glam-datasets-from-gov-portals/). Copy the url of the CSV file and paste it into the `.read_csv()` cell at the top. Of course, the column names are likely to be different, so you'll have to adjust them to suit.